In [ ]:
# suppress tensorflow logging, usually not useful unless you are having problems with tensorflow or accessing gput
# it seems necessary to have this environment variable set before tensorflow is imported, or else it doesn't take effect
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Check GPU Status / Availability

On linux systems you can and should use the `nvidia-smi` tool to check that the gpu is visible, is active and has drivers installed.  You can run the command from a terminal like the following cell.

I also find the following commands useful to monitor the gpu performance from the command line

```
# use watch so basic nvidia-smi redraws at top of screen each second
$ watch -n 1 nvidia-smi

# nvtop is basiclly like top for nvidia gpu
$ sudo apt install nvtop
$ nvtop

# nvitop is similar, gives about same information, but some may prefer this one
$ sudo apt install nvitop
$ nvitop
```

In [ ]:
!nvidia-smi

# GPU to TF/Keras Availability

We can check that tensorflow recognizes the presence of a GPU device as follows.

In [ ]:
print('Available Devices : ', tf.config.list_physical_devices())
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

# Test Training Performance with only CPU

We can compare performance with and without the gpu.  First we will train a model using only the cpu and record time it takes to train model.  We just train a simple dens NN on mnist data to test 
performance on fitting a model.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

with tf.device('/CPU:0'):
    model_cpu_base = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(1024, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
    ])


In [ ]:
def fit_cpu():
    with tf.device('/CPU:0'):
        model_cpu = tf.keras.models.clone_model(model_cpu_base)
        model_cpu.compile(optimizer='adam',
                      loss=loss_fn,
                      metrics=['accuracy'])
        model_cpu.fit(x_train, y_train, epochs=5)
        print('=' * 70)
        
%timeit fit_cpu()

# Test Training Performance with GPU

Now do same model and fit it on the gpu device.

In [ ]:
# recreate a random starting model and compile it
with tf.device('/GPU:0'):
    model_gpu_base = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(1024, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
    ])


In [ ]:
def fit_gpu():
    with tf.device('/GPU:0'):
        model_gpu = tf.keras.models.clone_model(model_gpu_base)
        model_gpu.compile(optimizer='adam',
                      loss=loss_fn,
                      metrics=['accuracy'])
        model_gpu.fit(x_train, y_train, epochs=5)
        print('=' * 70)

%timeit fit_gpu()